<a href="https://colab.research.google.com/github/loriboi/DecisionScienceOneWay/blob/main/codice_k1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
!pip install docplex
!pip install cplex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [61]:
#inizializzazione degli elementi

from docplex.mp.model import Model
import itertools
import numpy as np
import matplotlib.pyplot as plt
from random import *
import math

nodi = [0,1,2,3] # nodi: stazioni + deposito
periodo = [0,1,2,3]
nodi_t = [(i,t) for i in nodi for t in periodo]

archi = [(i, j) for i in nodi for j in nodi if(i!=j) ] # tutti gli archi
archi_w = [(i, j, t, t+1) for i in nodi for j in nodi if(i==j) for t in periodo] # archi in waiting(tutti i nodi sono uguali)
archi_m = [(i, j, t, t+1) for i in nodi for j in nodi if(i!=j) for t in periodo] # archi in motorcycle driving(tutti)
#da aggiungere dopo il deposito
archi_c = [(i, j, t,t+1) for i in nodi for j in nodi if(i!=j) for t in periodo] # archi in car driving(tutti tranne quelli che passano per il deposito)

c_m=0.8 #costo di ogni arco in monopattino
c_c=1.2 #costo di ogni arco in macchina
c_d=3
c_s=3 #costo per quando non può essere spostata una macchina in una stazione i per sovrapparcheggiamento

x_a_inizio=[] #numero di macchine presenti nel parcheggio i nel periodo t-1
for i in nodi:
  x=randint(1,8)
  x_a_inizio.append(x)

dimensione_s_i_t=len(nodi)*len(periodo)
s_i = [(i,t) for i in nodi for t in periodo] #richiesta di tutti i nodi
d_i = [(i,t) for i in nodi for t in periodo] #offerta di tutti i nodi


s_i_t=[]
for j in s_i:
  x=randint(1,6)
  s_i_t.append(x)


d_i_t=[]
for j in d_i:
  x=randint(1,6)
  d_i_t.append(x)

#per ora tutte impostate a 6
p_i=6 # numero massimo di parcheggi nella stazione i


In [62]:
d_i_t

[3, 4, 3, 2, 6, 5, 1, 4, 4, 2, 3, 3, 6, 1, 5, 4]

In [63]:
s_i_t

[3, 4, 6, 1, 1, 6, 1, 4, 3, 5, 4, 6, 4, 2, 2, 1]

In [64]:
#variabili decisionali

model = Model(name='Esercizio')

w = model.binary_var_dict(archi_w, name='w') #1 waiting
m = model.binary_var_dict(archi_m, name='m') #1 monopattino
c = model.binary_var_dict(archi_c,name='c') #1 macchina
x_a = model.integer_var_dict(nodi_t, name='x_a') #numero di macchine che possono essere prese dalla stazione i
x_d = model.integer_var_dict(nodi_t, name='x_d') #numero di macchine che non possono essere prenotate perchè devono essere spostate dal gestore
x_s = model.integer_var_dict(nodi_t, name='x_s') #numero di macchine che non possono essere riportate alla stazione i per il sovrannumero di macchine in quella stazione i


In [65]:

#I VINCOLO DOPO LE VARIABILI DECISIONALI
#vincolo per controllare che nel primo periodo il gestore k può o aspettare o muoversi   

model.add_constraint(sum(w[(i,i,0,1)] for i in nodi for j in nodi if ((i,i,0,1) in w))
+sum(m[(i,j,0,1)] for i in nodi for j in nodi if (i!=j) and ((i,j,0,1) in m))
+sum(c[(i,j,0,1)] for i in nodi for j in nodi if (i!=j) and ((i,j,0,1) in c))==1)

#II VINCOLO DOPO LE VARIABILI DECISIONALI

for t in periodo:
    for j in nodi:
      if(t>0 and t<4):
        model.add_constraint((w[(i,i,t-1,t)]+sum(m[(i,j,t-1,t)] for i in nodi if((i,j,t-1,t) in m) and (i!=j))
    +sum(c[(i,j,t-1,t)] for i in nodi if((i,j,t-1,t) in c) and (i!=j)))==
     (w[(i,i,t,t+1)]+sum(m[(j,i,t,t+1)] for i in nodi if((j,i,t,t+1) in m) and (i!=j))
         +sum(c[(j,i,t,t+1)] for i in nodi if((j,i,t,t+1) in c)and (i!=j))))

#III VINCOLO DOPO LE VARIABILI DECISIONALI
#spostamento dal penultimo all'ultimo
model.add_constraint(sum(w[(i,i,periodo[-2]-1,periodo[-1]-1)] for i in nodi for j in nodi if ((i,i,periodo[-2]-1,periodo[-1]-1) in w))
+sum(m[(i,j,periodo[-2]-1,periodo[-1]-1)] for i in nodi for j in nodi if (i!=j) and ((i,j,periodo[-2]-1,periodo[-1]-1) in m))
+sum(c[(i,j,periodo[-2]-1,periodo[-1]-1)] for i in nodi for j in nodi if (i!=j) and ((i,j,periodo[-2]-1,periodo[-1]-1) in c))==1)

#IV VINCOLO 
count=0
for t in periodo:
    for i in nodi:
      if (t!=0):
        count=count+1
        x_a[(i,t)]=x_a[(i,t-1)]+sum(c[(j,i,t-1,t)] for j in nodi if(i!=j)and (j,i,t-1,t) in c)
        -sum(c[(i,j,t,t+1)] for j in nodi if(i!=j) and (i,j,t,t+1) in c)
        -x_s[(i,t)]+s_i_t[count-1]-d_i_t[count-1]

#V VINCOLO è basato sulla manutenzione delle auto

#VI VINCOLO
count=0
#vincolo dove per ogni stazione la somma del numero di macchine 
for t in periodo:
  for i in nodi:
      if (t<3):
        count=count+1
        model.add_constraint(x_a[(i,t)]+s_i_t[count]-x_s[(i,t+1)] <= p_i)

#VII VINCOLO
count=0
#vincolo dove per ogni stazione la somma del numero di macchine 
for t in periodo:
  for i in nodi:
    if(t<3):
      count=count+1
      model.add_constraint(x_s[(i,t)] <= s_i_t[count])

#VIII VINCOLO
#vincolo dove per ogni stazione la somma del numero di macchine 
count=0
for t in periodo:
  for i in nodi:
    if(t<3):
      count=count+1
      model.add_constraint(x_d[(i,t)] <= d_i_t[count])


#obiettivo-seguendo il paper
model.minimize(sum(c_m*m[(i,j,t,t+1)]+c_c*c[(i,j,t,t+1)] for t in periodo for i in nodi for j in nodi if (i!=j) and ((i,j,t,t+1) in m))
+c_d*sum(x_d[(i,t)] for t in periodo for i in nodi)+c_s*sum(x_s[(i,t)] for t in periodo for i in nodi))

soluzione=model.solve(log_output="true")
print(model.solve_details.status)

Version identifier: 22.1.1.0 | 2023-02-11 | 22d6266e5
CPXPARAM_Read_DataCheck                          1
Tried aggregator 6 times.
MIP Presolve eliminated 36 rows and 146 columns.
MIP Presolve modified 2 coefficients.
Aggregator did 14 substitutions.
All rows and columns eliminated.
Presolve time = 0.01 sec. (0.48 ticks)

Root node processing (before b&c):
  Real time             =    0.01 sec. (0.50 ticks)
Parallel b&c, 2 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.01 sec. (0.50 ticks)
integer optimal solution


In [66]:
for t in periodo:
  print(f"\n\nPERIODO {t+1}\n")
  for i in nodi:
    for j in nodi:
      if(i!=j):
        if(soluzione.get_value(m[(i,j,t,t+1)])!=0):
          print(f"monopattino:[{m[(i,j,t,t+1)]}] = {soluzione.get_value(m[(i,j,t,t+1)])}")
  for i in nodi:
    for j in nodi:
      if(i!=j):
        if(soluzione.get_value(c[(i,j,t,t+1)])!=0):
          print(f"car:[{c[(i,j,t,t+1)]}] = {soluzione.get_value(c[(i,j,t,t+1)])}")
  for i in nodi:
        if(soluzione.get_value(w[(i,i,t,t+1)])!=0):
          print(f"waiting:[{w[(i,j,t,t+1)]}] = {soluzione.get_value(w[(i,i,t,t+1)])}")




PERIODO 1

monopattino:[m_3_0_0_1] = 1.0


PERIODO 2

monopattino:[m_0_2_1_2] = 1.0


PERIODO 3

monopattino:[m_2_0_2_3] = 1.0


PERIODO 4

monopattino:[m_0_1_3_4] = 1.0
